# 인공지능 작사가

### I 다음 am을 쓰면 반 이상은 맞더라

In [ ]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


## 데이터 다듬기

In [ ]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path =  '/content/drive/MyDrive/AIFFEL/EXP/[E6]text/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [ ]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 특수 문자 제거

In [ ]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [ ]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split())>15:
      continue
      
    corpus.append(preprocessed_sentence)

        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>',
 '<start> we are accounted poor citizens , the patricians good . <end>']

In [ ]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  141   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   49 ...    3    0    0]
 ...
 [   2  147 4524 ...    0    0    0]
 [   2   34   71 ...    3    0    0]
 [   2  930   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f2e4f66d750>


In [ ]:
print(tensor[:3, :10])

[[   2  141   40  919  140  604    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   49   43 1181  308    9  196   74    9]]


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [ ]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 141  40 919 140 604   4 124  24 110   5   3   0   0]
[141  40 919 140 604   4 124  24 110   5   3   0   0   0]


In [ ]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split( src_input, src_input, test_size=0.2, random_state=2)

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

print(steps_per_epoch)

92


In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(units = 128, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(units = 128, return_sequences=True)
        self.rnn_3 = tf.keras.layers.LSTM(units = 128, return_sequences=True)
        self.rnn_4 = tf.keras.layers.LSTM(units = 128 , return_sequences=True)
        self.drop  = tf.keras.layers.Dropout(0.5)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.rnn_3(out)
        out = self.rnn_4(out)
        out = self.drop(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[ 3.56553710e-06, -3.66374024e-06,  3.33420940e-06, ...,
          3.91724507e-06, -9.38156177e-07, -7.31692978e-07],
        [ 3.98815064e-06, -6.37040330e-06,  1.61752014e-05, ...,
          9.90091394e-06, -8.30023782e-06, -2.67064343e-06],
        [-1.16395277e-05, -5.70868042e-06,  3.27879352e-05, ...,
          1.17832597e-05, -2.12120394e-05, -1.53780329e-05],
        ...,
        [-6.08520000e-04,  3.10900563e-04,  5.96927421e-05, ...,
         -1.29475971e-04, -1.30081215e-04, -1.73603432e-04],
        [-6.19922474e-04,  3.75330361e-04,  5.34186147e-05, ...,
         -1.14704431e-04, -1.19941440e-04, -1.56632115e-04],
        [-6.20294188e-04,  4.28811210e-04,  4.87280377e-05, ...,
         -9.28123627e-05, -1.00792269e-04, -1.48267573e-04]],

       [[ 3.56553710e-06, -3.66374024e-06,  3.33420940e-06, ...,
          3.91724507e-06, -9.38156177e-07, -7.31692978e-07],
        [ 1.68586539e-05, -1.51113109e-05,  1.

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  197120    
                                                                 
 lstm_1 (LSTM)               multiple                  131584    
                                                                 
 lstm_2 (LSTM)               multiple                  131584    
                                                                 
 lstm_3 (LSTM)               multiple                  131584    
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense (Dense)               multiple               

In [ ]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
92/92 [==============================] - 10s 20ms/step - loss: 5.5931
Epoch 2/30
92/92 [==============================] - 2s 19ms/step - loss: 4.7751
Epoch 3/30
92/92 [==============================] - 2s 19ms/step - loss: 4.7516
Epoch 4/30
92/92 [==============================] - 2s 19ms/step - loss: 4.6562
Epoch 5/30
92/92 [==============================] - 2s 16ms/step - loss: 4.5157
Epoch 6/30
92/92 [==============================] - 2s 16ms/step - loss: 4.4391
Epoch 7/30
92/92 [==============================] - 2s 16ms/step - loss: 4.4112
Epoch 8/30
92/92 [==============================] - 2s 16ms/step - loss: 4.3986
Epoch 9/30
92/92 [==============================] - 2s 16ms/step - loss: 4.2776
Epoch 10/30
92/92 [==============================] - 2s 16ms/step - loss: 4.1293
Epoch 11/30
92/92 [==============================] - 2s 16ms/step - loss: 4.0775
Epoch 12/30
92/92 [==============================] - 2s 16ms/step - loss: 4.0491
Epoch 13/30
92/92 [=================

In [ ]:
results = model.evaluate(enc_val,  dec_val, verbose=2)

print(results)

148/148 - 3s - loss: 6.0263 - 3s/epoch - 17ms/step
6.026282787322998


In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he , the <unk> , and the <unk> , <end> '

# 프로젝트 : 멋진 작사가 만들기

In [1]:
import glob
import tensorflow

!pip list | grep glob
print(tensorflow.__version__)

glob2                         0.7
2.8.0


데이터 읽기

In [2]:
import glob
import os

txt_file_path = '/content/drive/MyDrive/AIFFEL/EXP/*text/data/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
# txt_list
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['THE QUEEN _of_ HEARTS', '', '']


데이터 정제

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 10: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

THE QUEEN _of_ HEARTS
    The Queen of Hearts she made some tarts,
      All on a summer's day;
    The Knave of Hearts he stole those tarts,
      And took them clean away.
    The King of Hearts called for those tarts,
    And beat the Knave full sore.


In [4]:
import re

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> the queen of hearts <end>',
 '<start> the queen of hearts she made some tarts , <end>',
 '<start> all on a summer s day <end>',
 '<start> the knave of hearts he stole those tarts , <end>',
 '<start> and took them clean away . <end>',
 '<start> the king of hearts called for those tarts , <end>',
 '<start> and beat the knave full sore . <end>',
 '<start> the knave of hearts brought back those tarts , <end>',
 '<start> and vowed he d steal no more . <end>',
 '<start> saint swithin s day <end>']

In [6]:
import tensorflow as tf

# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 15000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 16)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   6 818 ...   0   0   0]
 [  2   6 818 ...   0   0   0]
 [  2  24  18 ...   0   0   0]
 ...
 [  2   5  90 ...   0   0   0]
 [  2   9 157 ...   0   0   0]
 [  2 160  15 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fb815a85fd0>


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


모델 만들기

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2   6 818  19 953   3   0   0   0   0   0   0   0   0   0]
[  6 818  19 953   3   0   0   0   0   0   0   0   0   0   0]


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

# dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
# dataset = dataset.shuffle(BUFFER_SIZE)
# dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
# dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 15), dtype=tf.int32, name=None), TensorSpec(shape=(256, 15), dtype=tf.int32, name=None))>

In [10]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split( src_input, tgt_input, test_size=0.2, random_state=2)

train data

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 15), dtype=tf.int32, name=None), TensorSpec(shape=(256, 15), dtype=tf.int32, name=None))>

test data

In [12]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 15), dtype=tf.int32, name=None), TensorSpec(shape=(256, 15), dtype=tf.int32, name=None))>

In [13]:
# class TextGenerator(tf.keras.Model):
#     def __init__(self, vocab_size, embedding_size, hidden_size):
#         super().__init__()
        
#         self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
#         self.rnn_1 = tf.keras.layers.LSTM(units = 128, return_sequences=True)
#         self.rnn_2 = tf.keras.layers.LSTM(units = 64, return_sequences=True)
#         self.rnn_3 = tf.keras.layers.LSTM(units = 64, return_sequences=True)
#         self.rnn_4 = tf.keras.layers.LSTM(units = 64 , return_sequences=True)
#         self.rnn_5 = tf.keras.layers.LSTM(units = 32 , return_sequences=True)
#         self.drop  = tf.keras.layers.Dropout(0.5)
#         self.linear = tf.keras.layers.Dense(vocab_size)
        
#     def call(self, x):
#         out = self.embedding(x)
#         out = self.rnn_1(out)
#         out = self.rnn_2(out)
#         out = self.rnn_3(out)
#         out = self.rnn_4(out)
#         out = self.rnn_5(out)
#         out = self.drop(out)
#         out = self.linear(out)
        
#         return out
    
# embedding_size = 256
# hidden_size = 1024
# model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

Model : Embedding layer - LSTM - LSTM - Dense

In [24]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.rnn_3(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [25]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 15, 20001), dtype=float32, numpy=
array([[[ 6.69463952e-06, -1.82858457e-05, -1.39801223e-05, ...,
          9.91448542e-06,  2.91215765e-05,  3.11503427e-05],
        [-1.05817126e-05,  3.48311869e-06, -2.57709198e-05, ...,
          4.57393762e-05,  5.86533715e-05,  5.90081072e-05],
        [-6.98880613e-05,  6.63547107e-05, -6.65963162e-05, ...,
          5.94711601e-05,  5.05249591e-05,  7.12204856e-05],
        ...,
        [ 1.10658782e-03,  8.09071935e-04, -4.14307695e-04, ...,
          5.48216398e-04, -1.22786034e-03,  5.03679854e-04],
        [ 1.41406560e-03,  8.09527352e-04, -4.76389978e-04, ...,
          6.02879270e-04, -1.40215014e-03,  6.12686155e-04],
        [ 1.70619704e-03,  7.91847240e-04, -5.41874149e-04, ...,
          6.38786179e-04, -1.55658193e-03,  7.24436191e-04]],

       [[ 6.69463952e-06, -1.82858457e-05, -1.39801223e-05, ...,
          9.91448542e-06,  2.91215765e-05,  3.11503427e-05],
        [ 5.78785039e-05,  7.83732321e-06, -2

In [26]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  10240512  
                                                                 
 lstm_2 (LSTM)               multiple                  6295552   
                                                                 
 lstm_3 (LSTM)               multiple                  8392704   
                                                                 
 lstm_4 (LSTM)               multiple                  8392704   
                                                                 
 dense_1 (Dense)             multiple                  20501025  
                                                                 
Total params: 53,822,497
Trainable params: 53,822,497
Non-trainable params: 0
_________________________________________________________________


In [27]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
549/549 [==============================] - 99s 175ms/step - loss: 3.7106 - val_loss: 3.3655
Epoch 2/10
549/549 [==============================] - 95s 174ms/step - loss: 3.2128 - val_loss: 3.1460
Epoch 3/10
549/549 [==============================] - 95s 174ms/step - loss: 3.0349 - val_loss: 3.0337
Epoch 4/10
549/549 [==============================] - 95s 174ms/step - loss: 2.9077 - val_loss: 2.9480
Epoch 5/10
549/549 [==============================] - 95s 174ms/step - loss: 2.8040 - val_loss: 2.8835
Epoch 6/10
549/549 [==============================] - 95s 174ms/step - loss: 2.7156 - val_loss: 2.8351
Epoch 7/10
549/549 [==============================] - 95s 173ms/step - loss: 2.6361 - val_loss: 2.7940
Epoch 8/10
549/549 [==============================] - 95s 174ms/step - loss: 2.5621 - val_loss: 2.7610
Epoch 9/10
549/549 [==============================] - 95s 174ms/step - loss: 2.4925 - val_loss: 2.7329
Epoch 10/10
549/549 [==============================] - 95s 173ms/step - l

In [29]:
results = model.evaluate(enc_val,  dec_val, verbose=2)

print(results)

1099/1099 - 14s - loss: 2.7104 - 14s/epoch - 13ms/step
2.710362434387207


In [30]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [31]:
generate_text(model, tokenizer, init_sentence="<start> sky ", max_len=20)

'<start> sky is the limit and you know that you can <end> '

1.Train data와 evalution data로 분리하지 않고 source 데이타와 target데이터로 분리하였더니 매우 높은 평가 값이 나왔지만 evalution 데이터를 넣었더니 매우 낮은 값을 얻을 수 있었다.  

2. 데이터 수정하는 과정에서 dataset을 tensor_slice, shuffle, 그리고 batch 하는 과정에서 동일한 이름의 dataset 이름을 적어 놓아야 하는데 그렇지 못하여 shape(256, 16)이어야 하는데 다른 dataset으로 하였는데 기존 데이터와 겹치면서 (256, 256, 16)으로 나와 model.fit이 실행이 안 되었다. 한 줄 한 줄 하나씩 보는 것이 중요한 것을 알 수 있었다.